In [2]:
import os
import pandas as pd
import numpy as np
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
from sklearn.model_selection import cross_val_predict
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score

In [4]:
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error

In [5]:
output_dir_ccle = 'C:\\Users\\MONSTER\\Desktop\\TEZ SON\\CCLE\\OUTPUT - CCLE-GDSC'
output_dir_gdsc = 'C:\\Users\\MONSTER\\Desktop\\TEZ SON\\GDSC\\output'

In [6]:
os.chdir(output_dir_ccle)

In [7]:
Histologies = [item1 for item1 in os.listdir() if os.path.isdir(item1)]

In [7]:
Histology_dict = dict()
Drug_dict = dict()
IC50_predict = dict()

for histology in Histologies:
    os.chdir(os.path.join(output_dir_ccle, histology))
    if len(os.listdir()) > 1:
        Drugs_ccle = [item1 for item1 in os.listdir() if os.path.isdir(item1)]
        
        os.chdir(os.path.join(output_dir_gdsc, histology))
        if len(os.listdir()) > 1:
            Drugs_gdsc = [item1 for item1 in os.listdir() if os.path.isdir(item1)]
            
            common_drug_ccle_gdsc = set(Drugs_ccle).intersection(Drugs_gdsc)
            if len(common_drug_ccle_gdsc) > 0:
                
                for drug in common_drug_ccle_gdsc:
                    
                    expression_data = pd.read_csv(os.path.join(output_dir_ccle,histology,drug,drug+" "+histology+" "+"- Expression Data(IC50).tsv"), sep = ",")
                    gdsc_train_data = pd.read_csv(os.path.join(output_dir_gdsc,histology,drug,drug+" "+histology+" "+"- GDSC Histology Project.tsv"), sep = "\t")
                    
                    if len(expression_data.columns) > 3 and len(expression_data['Cell Line']) > 20 and len(gdsc_train_data['Cell Line']) > 20:
                        
                        x_ccle = expression_data.drop(['Unnamed: 0', 'Cell Line', 'LN_IC50'], axis=1)
                        y_ccle = expression_data['LN_IC50']
                        
                        x_gdsc_1 = gdsc_train_data.drop(['Cell Line','GDSC\nTissue descriptor 1','LN_IC50','Drug'], axis=1)
                        y_gdsc = gdsc_train_data['LN_IC50']
                    
                        common_features_ccle_gdsc = set(x_ccle.columns).intersection(x_gdsc_1.columns)
                        if len(common_features_ccle_gdsc) > 0:
                            
                            features_name = common_features_ccle_gdsc
                            sample_number_gdsc_drug = len(gdsc_train_data['Cell Line'])
                        
                            x_ccle = x_ccle[list(features_name)]
                            #y_ccle = boruta_drug_data_ccle['log IC50 (uM)']
                                
                            x_gdsc = x_gdsc_1[list(features_name)]
                            #y_gdsc = boruta_drug_data_gdsc[drug]
                                
                            features_name_number= len((x_gdsc).columns)
                                
                            x_ccle = np.array(x_ccle)
                            y_ccle = np.array(y_ccle)
                            x_gdsc = np.array(x_gdsc)
                            y_gdsc = np.array(y_gdsc)
                        
                        
                            # MODEL OLUŞTURUYORUZ
                                
                            # LOOCV (Leave-One-Out Cross-Validation)
                            loo = LeaveOneOut()
                            svr_model = SVR(kernel='rbf')
                            scores_loocv = cross_val_score(svr_model, x_ccle, y_ccle, cv=loo, scoring='neg_mean_squared_error')
                            mean_score_loocv = scores_loocv.mean()

                            # k-fold kullanılıyor.
                            kfold = KFold(n_splits=2, random_state=1, shuffle = True)
                            scores_kfold = cross_val_score(svr_model, x_ccle, y_ccle, cv=kfold, scoring='neg_mean_squared_error')
                            mean_score_kfold = scores_kfold.mean()

                            svr_model.fit(x_ccle, y_ccle)

                                
                            # Tahmin yapılıyor.
                            y_gdsc_pred = svr_model.predict(x_gdsc)                      

                            print(histology)
                            print(f"\nDrug : SVR is running for {drug}", flush=True)
            
                
                            r_squared = r2_score(y_gdsc, y_gdsc_pred)
                            pearson_correlation, p_value_pearsonr = pearsonr(y_gdsc_pred, y_gdsc)
                            corr_spearmanr, p_value_spearmanr = spearmanr(y_gdsc_pred, y_gdsc)
                            rmse_val = np.sqrt(mean_squared_error(y_gdsc_pred, y_gdsc))
                
                            features_name = list(features_name)
                            result_table_drug = pd.DataFrame({
                            'Tissue': [histology],
                            'Drug': [drug],
                            'R_Square': [r_squared],
                            'Pearson_Correlation_Train': [pearson_correlation],
                            'features_name': [features_name],
                            'features_name_number': [features_name_number],
                            'sample_number_gdsc_drug':[sample_number_gdsc_drug],    
                            'Pearson_Correlation_P_Value_Train': [p_value_pearsonr],
                            'Spearman_Correlation_Train': [corr_spearmanr],
                            'Spearman_Correlation_P_Value_Train': [p_value_spearmanr],
                            'RMSE_Train': [rmse_val]
                             })
            

                            Drug_dict[drug] = result_table_drug
                
                            # Save results to files
                            os.mkdir(os.path.join(output_dir_gdsc, histology, drug, "SVR - Regression"))
                            result_table_drug.to_csv(os.path.join(output_dir_gdsc, histology, drug, "SVR - Regression", drug + "- Result_SVR_Reg.tsv"), 
                                     sep='\t', index=False, quoting=False)
                            IC50_predict[drug] = pd.DataFrame({'Actual_IC50_Data': y_gdsc, 'Predicted_IC50_Data': y_gdsc_pred})
                            IC50_predict[drug].to_csv(os.path.join(output_dir_gdsc, histology, drug, "SVR - Regression", drug + "- IC50_Result.tsv"), 
                                            sep='\t', index=False, quoting=False)
                       
                    
                    
                    
    if Drug_dict:
        Histology_dict[histology] = pd.concat(Drug_dict.values())  # Her bir histoloji için ilaç sonuçlarını birleştirin

# Tüm histolojilerin sonuçlarını birleştirin ve dosyalara kaydedin
if Histology_dict:
    All_SVR_Result = pd.concat(Histology_dict.values())

    


aero_dig_tract

Drug : SVR is running for PD0325901
aero_dig_tract

Drug : SVR is running for AZD6244
aero_dig_tract

Drug : SVR is running for 17-AAG
breast

Drug : SVR is running for PD0325901
breast

Drug : SVR is running for AZD6244
breast

Drug : SVR is running for 17-AAG
digestive_system

Drug : SVR is running for PD0325901
digestive_system

Drug : SVR is running for AZD6244
digestive_system

Drug : SVR is running for 17-AAG
digestive_system

Drug : SVR is running for PLX-4720
digestive_system

Drug : SVR is running for Nilotinib
kidney

Drug : SVR is running for PD0325901
kidney

Drug : SVR is running for AZD6244
kidney

Drug : SVR is running for 17-AAG
kidney

Drug : SVR is running for Nilotinib
large_intestine

Drug : SVR is running for PD0325901
large_intestine

Drug : SVR is running for AZD6244
large_intestine

Drug : SVR is running for 17-AAG
large_intestine

Drug : SVR is running for Nilotinib
leukemia

Drug : SVR is running for Paclitaxel
leukemia

Drug : SVR is running f

In [11]:
All_SVR_Result = All_SVR_Result.sort_values(by='R_Square', ascending=False)
All_SVR_Result.to_excel('C:\\Users\\MONSTER\\Desktop\\TEZ SON\\CCLE\\ML RESULT\\SVR Regression_CCLE_GDSC.xlsx', index=False)

In [ ]:
Histology_dict = dict()
Drug_dict = dict()
IC50_predict = dict()

for histology in Histologies:
    os.chdir(os.path.join(output_dir_ccle, histology))
    if len(os.listdir()) > 1:
        Drugs_ccle = [item1 for item1 in os.listdir() if os.path.isdir(item1)]
        
        os.chdir(os.path.join(output_dir_gdsc, histology))
        if len(os.listdir()) > 1:
            Drugs_gdsc = [item1 for item1 in os.listdir() if os.path.isdir(item1)]
            
            common_drug_ccle_gdsc = set(Drugs_ccle).intersection(Drugs_gdsc)
            if len(common_drug_ccle_gdsc) > 0:
                
                for drug in common_drug_ccle_gdsc:
                    
                    expression_data = pd.read_csv(os.path.join(output_dir_ccle,histology,drug,drug+" "+histology+" "+"- Expression Data(IC50).tsv"), sep = ",")
                    gdsc_train_data = pd.read_csv(os.path.join(output_dir_gdsc,histology,drug,drug+" "+histology+" "+"- GDSC Histology Project.tsv"), sep = "\t")
                    
                    if len(expression_data.columns) > 3 and len(expression_data['Cell Line']) > 20 and len(gdsc_train_data['Cell Line']) > 20:
                        
                        x_ccle = expression_data.drop(['Unnamed: 0', 'Cell Line', 'LN_IC50'], axis=1)
                        y_ccle = expression_data['LN_IC50']
                        
                        x_gdsc_1 = gdsc_train_data.drop(['Cell Line','GDSC\nTissue descriptor 1','LN_IC50','Drug'], axis=1)
                        y_gdsc = gdsc_train_data['LN_IC50']
                    
                        common_features_ccle_gdsc = set(x_ccle.columns).intersection(x_gdsc_1.columns)
                        if len(common_features_ccle_gdsc) > 0:
                            
                            features_name = common_features_ccle_gdsc
                            sample_number_gdsc_drug = len(gdsc_train_data['Cell Line'])
                        
                            x_ccle = x_ccle[list(features_name)]
                            #y_ccle = boruta_drug_data_ccle['log IC50 (uM)']
                                
                            x_gdsc = x_gdsc_1[list(features_name)]
                            #y_gdsc = boruta_drug_data_gdsc[drug]
                                
                            features_name_number= len((x_gdsc).columns)
                                
                            x_ccle = np.array(x_ccle)
                            y_ccle = np.array(y_ccle)
                            x_gdsc = np.array(x_gdsc)
                            y_gdsc = np.array(y_gdsc)
                        
                        
                            # MODEL OLUŞTURUYORUZ
                            
                            param_grid = {
                           'C': [0.1, 1, 10, 100],
                           'epsilon': [0.01, 0.1, 1],
                           'kernel': ['linear', 'poly', 'rbf'],
                           'gamma': ['scale', 'auto']
                            }
                            
                            grid_search = GridSearchCV(SVR(), param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
                            grid_search.fit(x_ccle, y_ccle)
                            
                            best_params = grid_search.best_params_
                            svr_optimized = SVR(**best_params)
                            svr_optimized.fit(x_ccle, y_ccle)
                            
                            y_gdsc_pred = svr_optimized.predict(x_gdsc)
                            
                            mse_optimized = mean_squared_error(y_gdsc, y_gdsc_pred)
                            r_squared = r2_score(y_gdsc, y_gdsc_pred)


                                


                                
                            # Tahmin yapılıyor.

                            print(histology)
                            print(f"\nDrug : SVR is running for {drug}", flush=True)
            
                
                            pearson_correlation, p_value_pearsonr = pearsonr(y_gdsc_pred, y_gdsc)
                            corr_spearmanr, p_value_spearmanr = spearmanr(y_gdsc_pred, y_gdsc)
                            rmse_val = np.sqrt(mean_squared_error(y_gdsc_pred, y_gdsc))
                
                            features_name = list(features_name)
                            result_table_drug = pd.DataFrame({
                            'Tissue': [histology],
                            'Drug': [drug],
                            'R_Square': [r_squared],
                            'Pearson_Correlation_Train': [pearson_correlation],
                            'features_name': [features_name],
                            'features_name_number': [features_name_number],
                            'sample_number_gdsc_drug':[sample_number_gdsc_drug],    
                            'Pearson_Correlation_P_Value_Train': [p_value_pearsonr],
                            'Spearman_Correlation_Train': [corr_spearmanr],
                            'Spearman_Correlation_P_Value_Train': [p_value_spearmanr],
                            'RMSE_Train': [rmse_val]
                             })
            

                            Drug_dict[drug] = result_table_drug
                
                            # Save results to files
                            os.mkdir(os.path.join(output_dir_gdsc, histology, drug, "SVR - Regression_deneme hyper"))
                            result_table_drug.to_csv(os.path.join(output_dir_gdsc, histology, drug, "SVR - Regression_deneme hyper", drug + "- Result_SVR_Reg_deneme.tsv"), 
                                     sep='\t', index=False, quoting=False)
                            IC50_predict[drug] = pd.DataFrame({'Actual_IC50_Data': y_gdsc, 'Predicted_IC50_Data': y_gdsc_pred})
                            IC50_predict[drug].to_csv(os.path.join(output_dir_gdsc, histology, drug, "SVR - Regression_deneme hyper", drug + "- IC50_Result.tsv"), 
                                            sep='\t', index=False, quoting=False)
                       
                    
                    
                    
    if Drug_dict:
        Histology_dict[histology] = pd.concat(Drug_dict.values())  # Her bir histoloji için ilaç sonuçlarını birleştirin

# Tüm histolojilerin sonuçlarını birleştirin ve dosyalara kaydedin
if Histology_dict:
    All_SVR_Result = pd.concat(Histology_dict.values())

    


aero_dig_tract

Drug : SVR is running for AZD6244
aero_dig_tract

Drug : SVR is running for PD0325901


In [ ]:
All_SVR_Result = All_SVR_Result.sort_values(by='R_Square', ascending=False)
All_SVR_Result.to_excel('C:\\Users\\MONSTER\\Desktop\\TEZ SON\\CCLE\\ML RESULT\\SVR Regression_CCLE_GDSC DENEME hyper.xlsx', index=False)

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score